<a href="https://colab.research.google.com/github/ashibullah/Romanian-Bangla-Sentiment-Analysis-NLP/blob/main/RomanianBanglaSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets --upgrade

In [3]:
from datasets import load_dataset

In [6]:
df = load_dataset("aplycaebous/BnSentMix" , split = "train")

In [9]:
print(df[20])

{'Sentence': 'Vloi bt cash out korte 15 takar jaigai 18.50 paisa nei', 'Label': 3}
